In [82]:
from classes.network import Network
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

def shuffle_and_split(X_train, y_train, batch_size=10):
    p = np.random.permutation(len(X_train))

    new_X_train = X_train[p]
    new_y_train = y_train[p]

    splitted_X_train = np.array_split(new_X_train, batch_size)
    splitted_y_train = np.array_split(new_y_train, batch_size)

    return splitted_X_train[:-1], splitted_y_train[:-1], splitted_X_train[-1], splitted_y_train[-1]

ONE_IN_FILES = [
    "data/1in_tanh.txt",
    "data/1in_linear.txt",
    "data/1in_cubic.txt",
    "data/1in_sine.txt"
]

In [6]:
network = Network(error="l2")
network.addLayer(3, inputSize=1, activation="tanh")
network.addLayer(1, activation="tanh")

ds = pd.read_csv("data/1in_tanh.txt", sep=r"\s+", header=None)
X_train, y_train = ds.iloc[:,:-1].to_numpy(), ds.iloc[:,1].to_numpy()

splitted_X_train, splitted_y_train = shuffle_and_split(X_train, y_train, batch_size=20)

p = np.random.permutation(len(X_train))

new_X_train = X_train[p][:10]
new_y_train = y_train[p][:10]


#network.train(new_X_train, new_y_train)
#network.train_with_pso(new_X_train, np.expand_dims(new_y_train, axis=1), iter_count=100)

perf = network.train_with_pso(X_train, y_train)

#for (x_batch,y_batch) in tqdm(zip(splitted_X_train,splitted_y_train)):
#    perf = network.train_with_pso(x_batch, y_batch)

Best_fitness: 0.24666885972889335


In [12]:
a[:-1]

[1, 2, 3]

# Shallow vs Deep neural networks

When training neural networks, we have to come up with a design that fits its intention. Here, we tried to find, given a certain dataset, which architecture was the most ideal.

In [16]:
DATA_FILES = [
    "data/1in_tanh.txt",
    "data/1in_linear.txt",
    "data/1in_cubic.txt",
    "data/1in_sine.txt",
    "data/2in_complex.txt",
    "data/2in_xor.txt"
]
def load_dataset(path):
    ds = pd.read_csv(path, sep=r"\s+", header=None)
    X_train, y_train = ds.iloc[:,:-1].to_numpy(), ds.iloc[:,1].to_numpy()
    return X_train, y_train

In [83]:
result_networks_n = {
    "1in_tan":{},
    "1in_linear":{},
    "1in_cubic":{},
    "1in_sine":{},
    "2in_complex":{},
    "2in_xor":{}
}

#batch:20,swarm:100,iter:10
for i, dataset in enumerate(result_networks_n.keys()):
    #Loading the dataset
    X_train, y_train = load_dataset(DATA_FILES[i])
    splitted_X_train, splitted_y_train, X_test, y_test = shuffle_and_split(X_train, y_train, batch_size=20)
    
    # First, creating a shallow network
    shallow = Network(error="l2")
    shallow.addLayer(1, inputSize=eval(dataset[0]), activation="tanh")
    print('-----------------------------')
    print(f"\tTraining shallow on {dataset}")
    
    start_shallow = time.time()
    shallow_train_perfs = []
    for (x_batch,y_batch) in zip(splitted_X_train,splitted_y_train):
        perf = shallow.train_with_pso(x_batch, y_batch)
        shallow_train_perfs.append(perf)
        
    shallow_test = shallow.test(X_test, y_test)
    print(f"Error on test: {shallow_test}")
    end_shallow = time.time()
        
    print(f"Time: {end_shallow - start_shallow}")
    
    start_deep = time.time()
    # Then, creating a deep network
    deep = Network(error="l2")
    deep.addLayer(3, inputSize=eval(dataset[0]), activation="tanh")
    deep.addLayer(3, activation="tanh")
    deep.addLayer(1, activation="tanh")
    print('-----------------------------')
    print(f"\tTraining deep on {dataset}")
    
    deep_train_perfs = []
    for (x_batch,y_batch) in zip(splitted_X_train,splitted_y_train):
        perf = deep.train_with_pso(x_batch, y_batch)
        deep_train_perfs.append(perf)
        
    deep_test = deep.test(X_test, y_test)
    print(f"Error on test: {deep_test}")
    end_deep = time.time()
    
    print(f"Time: {end_deep - start_deep}")
    
    result_networks_n[dataset]["shallow"] = {
        "model": shallow,
        "train_perfs": shallow_train_perfs,
        "test_perfs": shallow_test,
        "exec_time": end_shallow - start_shallow
    }
    
    result_networks_n[dataset]["deep"] = {
        "model": deep,
        "train_perfs": deep_train_perfs,
        "test_perfs": deep_test,
        "exec_time": end_deep - start_deep
    }
    
    

-----------------------------
	Training shallow on 1in_tan
Best_fitness: 0.2103871019794531
Best_fitness: 0.0455906612952442
Best_fitness: 0.29271410180362856
Best_fitness: 0.18206954732919806
Best_fitness: 0.4721164461194058
Best_fitness: 0.10384565845561244
Best_fitness: 0.24906490769168146
Best_fitness: 0.1093259270706779
Best_fitness: 0.15032911513211467
Best_fitness: 0.2927384913201504
Best_fitness: 0.18444690734727281
Best_fitness: 0.23764189535827832
Best_fitness: 0.40106839593079807
Best_fitness: 0.13539688845854153
Best_fitness: 0.5143778913755506
Best_fitness: 0.39654508876076144
Best_fitness: 0.5088323156783991
Best_fitness: 0.31362225851409686
Best_fitness: 0.3471119185104138
Error on test: 0.24356816493468614
-----------------------------
	Training deep on 1in_tan
Time: 238.13657689094543
Best_fitness: 0.05696864103106072
Best_fitness: 0.0094067055204669
Best_fitness: 0.046266394632666644
Best_fitness: 0.05565470036256337
Best_fitness: 0.0669014560737281
Best_fitness: 0.01

Best_fitness: 0.06411074635501066
Best_fitness: 0.00025126741317806317
Best_fitness: 0.07836699410592918
Best_fitness: 0.00021945374869083593
Best_fitness: 0.04373131947805849
Best_fitness: 0.12386312761792473
Best_fitness: 0.15222889150010802
Error on test: 0.11562780091810754
-----------------------------
	Training deep on 2in_xor
Time: 247.64712953567505
Best_fitness: 6.798917397277762e-05
Best_fitness: 5.744546015961237e-05
Best_fitness: 0.05069628041923681
Best_fitness: 0.09887257288325282
Best_fitness: 1.2753262957985491e-05
Best_fitness: 0.004435070956000231
Best_fitness: 0.056777631008383546
Best_fitness: 0.05604298359174935
Best_fitness: 1.8229327749940385e-05
Best_fitness: 0.031229293203202134
Best_fitness: 0.00017695738730108318
Best_fitness: 0.09088703790752355
Best_fitness: 0.05964144003691372
Best_fitness: 8.113910475508333e-06
Best_fitness: 0.09217590958246799
Best_fitness: 4.51585884149932e-05
Best_fitness: 0.10304915837567677
Best_fitness: 7.778583158472289e-07
Best_fi

In [84]:
for i, dataset in enumerate(result_networks.keys()):
    print(f"For dataset '{dataset}':")
    print("\tSHALLOW")
    print(f"Test perfs: {result_networks[dataset]['shallow']['test_perfs']}")
    print(f"Exec time: {result_networks[dataset]['shallow']['exec_time']}")
          
    print("\tDEEP")
    print(f"Test perfs: {result_networks[dataset]['deep']['test_perfs']}")
    print(f"Exec time: {result_networks[dataset]['deep']['exec_time']}")
    print("","-----------------\n")

For dataset '1in_tan':
	SHALLOW
Test perfs: 0.38824816006610957
Exec time: -0.0007770000001983135
	DEEP
Test perfs: 0.29896031900950215
Exec time: -0.00015849999954298255
 -----------------

For dataset '1in_linear':
	SHALLOW
Test perfs: 0.011827324932823966
Exec time: -0.0019315000008646166
	DEEP
Test perfs: 0.011349200814112427
Exec time: 5.539999983739108e-05
 -----------------

For dataset '1in_cubic':
	SHALLOW
Test perfs: 0.006408835749083383
Exec time: 0.0024451999997836538
	DEEP
Test perfs: 0.028563092705184194
Exec time: 0.00017110000044340268
 -----------------

For dataset '1in_sine':
	SHALLOW
Test perfs: 0.04884908371852646
Exec time: -1.1699999959091656e-05
	DEEP
Test perfs: 0.09984686398727441
Exec time: -2.0899999071843922e-05
 -----------------

For dataset '2in_complex':
	SHALLOW
Test perfs: 0.06502299035036482
Exec time: -0.0009059999993041856
	DEEP
Test perfs: 0.21276196445915163
Exec time: 0.00012910000077681616
 -----------------

For dataset '2in_xor':
	SHALLOW
Tes

In [79]:
X_train, y_train = load_dataset("data/2in_xor.txt")

y_pred = result_networks["2in_xor"]["shallow"]["model"].predict(X_train)

In [80]:
np.square(y_train-np.array(y_pred)).mean()

0.1307838517765549

## Trying to improve Deep architecture training by tuning PSO parameters such as swarm size and iterations

Giving PSO the right tools for it to reach better convergence is the reason that motivates these investigations ! 

In [ ]:
result_networks_custom = {
    "1in_tan":{},
    "1in_linear":{},
    "1in_cubic":{},
    "1in_sine":{},
    "2in_complex":{},
    "2in_xor":{}
}

#batch:50,swarm:100,iter:100
for i, dataset in enumerate(result_networks_n.keys()):
    #Loading the dataset
    X_train, y_train = load_dataset(DATA_FILES[i])
    splitted_X_train, splitted_y_train, X_test, y_test = shuffle_and_split(X_train, y_train, batch_size=50)
    
    # First, creating a shallow network
    shallow = Network(error="l2")
    shallow.addLayer(1, inputSize=eval(dataset[0]), activation="tanh")
    print('-----------------------------')
    print(f"\tTraining shallow on {dataset}")
    
    start_shallow = time.time()
    shallow_train_perfs = []
    for (x_batch,y_batch) in zip(splitted_X_train,splitted_y_train):
        perf = shallow.train_with_pso(x_batch, y_batch, iter_count=100)
        shallow_train_perfs.append(perf)
        
    shallow_test = shallow.test(X_test, y_test)
    print(f"Error on test: {shallow_test}")
    end_shallow = time.time()
    
    start_deep = time.time()
    # Then, creating a deep network
    deep = Network(error="l2")
    deep.addLayer(3, inputSize=eval(dataset[0]), activation="tanh")
    deep.addLayer(3, activation="tanh")
    deep.addLayer(1, activation="tanh")
    print('-----------------------------')
    print(f"\tTraining deep on {dataset}")
        
    print(f"Time: {end_shallow - start_shallow}")
    
    deep_train_perfs = []
    for (x_batch,y_batch) in zip(splitted_X_train,splitted_y_train):
        perf = deep.train_with_pso(x_batch, y_batch)
        deep_train_perfs.append(perf)
        
    deep_test = deep.test(X_test, y_test)
    print(f"Error on test: {deep_test}")
    end_deep = time.time()
    
    print(f"Time: {end_deep - start_deep}")
    
    result_networks_custom[dataset]["shallow"] = {
        "model": shallow,
        "train_perfs": shallow_train_perfs,
        "test_perfs": shallow_test,
        "exec_time": end_shallow - start_shallow
    }
    
    result_networks_custom[dataset]["deep"] = {
        "model": deep,
        "train_perfs": deep_train_perfs,
        "test_perfs": deep_test,
        "exec_time": end_deep - start_deep
    }
    
    

-----------------------------
	Training shallow on 1in_tan
Best_fitness: 0.36891633677912133
Best_fitness: 0.12000245007772414
Best_fitness: 2.8328908257249755e-05
Best_fitness: 0.0817705455604457
Best_fitness: 0.5686625791755207
Best_fitness: 0.5341732491910387
Best_fitness: 0.36738097802905334
Best_fitness: 0.20430529522477373
Best_fitness: 0.2785488108431602
Best_fitness: 0.3874664079920215
Best_fitness: 0.0004833183263129269
Best_fitness: 0.3493640531385038
Best_fitness: 0.12674348921277398
Best_fitness: 0.20045381899087994


In [ ]:
for i, dataset in enumerate(result_networks.keys()):
    print(f"For dataset '{dataset}':")
    print("\tSHALLOW")
    print(f"Test perfs: {result_networks_custom[dataset]['shallow']['test_perfs']}")
    print(f"Exec time: {result_networks_custom[dataset]['shallow']['exec_time']}")
          
    print("\tDEEP")
    print(f"Test perfs: {result_networks_custom[dataset]['deep']['test_perfs']}")
    print(f"Exec time: {result_networks_custom[dataset]['deep']['exec_time']}")
    print("","-----------------\n")

In [3]:
net1 = Network(error="l2")
net1.addLayer(1, inputSize=1, activation="identity")

net2 = Network(error="l2")
net2.addLayer(1, inputSize=1, activation="identity")

net3 = Network(error="l2")
net3.addLayer(3, inputSize=1, activation="identity")
net3.addLayer(1, activation="identity")

net4 = Network(error="l2")
net4.addLayer(3, inputSize=1, activation="identity")
net4.addLayer(1, inputSize=1, activation="identity")

nets = [net1,net2,net3,net4]

In [4]:
datasets = [
    pd.read_csv(filename, sep=r"\s+", header=None) for filename in ONE_IN_FILES
]

In [5]:
perfs = {
    "net_tan":{},
    "net_linear":{},
    "net_cubic":{},
    "net_sine":{}
}

for i, dataset in tqdm(enumerate(datasets)):
    X_train, y_train = dataset.iloc[:,:-1].to_numpy(), dataset.iloc[:,1].to_numpy()

    splitted_X_train, splitted_y_train = shuffle_and_split(X_train, y_train, batch_size=20)
    
    perfs[list(perfs.keys())[i]]["batch_size:20 swarmsize:100 itercount:100"] = []
    network = nets[i]
    for (x_batch,y_batch) in tqdm(zip(splitted_X_train,splitted_y_train)):
        perf = network.train_with_pso(x_batch, y_batch)
        perfs[list(perfs.keys())[i]]["batch_size:20 swarmsize:100 itercount:100"].append(perf)
        

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Best_fitness: [0.29270773]



1it [00:13, 13.46s/it]

Best_fitness: [0.37899608]



2it [00:25, 12.90s/it]

...


18it [05:54, 19.70s/it]

Best_fitness: [0.07890731]



19it [06:14, 19.73s/it]

Best_fitness: [0.09896203]



20it [06:33, 19.68s/it]
4it [21:05, 316.47s/it]


In [6]:
for i, dataset in tqdm(enumerate(datasets)):
    X_train, y_train = dataset.iloc[:,:-1].to_numpy(), dataset.iloc[:,1].to_numpy()

    splitted_X_train, splitted_y_train = shuffle_and_split(X_train, y_train, batch_size=10)
    
    perfs[list(perfs.keys())[i]]["batch_size:10 swarmsize:100 itercount:100"] = []
    network = nets[i]
    for (x_batch,y_batch) in tqdm(zip(splitted_X_train,splitted_y_train)):
        perf = network.train_with_pso(x_batch, y_batch)
        perfs[list(perfs.keys())[i]]["batch_size:10 swarmsize:100 itercount:100"].append(perf)
        

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Best_fitness: [0.31351796]



1it [00:22, 22.80s/it]

Best_fitness: [0.25526409]



2it [00:43, 22.23s/it]

...


8it [04:38, 34.67s/it]

Best_fitness: [0.13992826]



9it [05:12, 34.72s/it]

Best_fitness: [0.12097802]



10it [05:47, 34.74s/it]
4it [18:47, 281.98s/it]


In [7]:
for i, dataset in tqdm(enumerate(datasets)):
    X_train, y_train = dataset.iloc[:,:-1].to_numpy(), dataset.iloc[:,1].to_numpy()

    splitted_X_train, splitted_y_train = shuffle_and_split(X_train, y_train, batch_size=20)
    
    perfs[list(perfs.keys())[i]]["batch_size:20 swarmsize:200 itercount:100"] = []
    network = nets[i]
    for (x_batch,y_batch) in tqdm(zip(splitted_X_train,splitted_y_train)):
        perf = network.train_with_pso(x_batch, y_batch, swarm_size=200)
        perfs[list(perfs.keys())[i]]["batch_size:20 swarmsize:200 itercount:100"].append(perf)
 

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Best_fitness: [0.25759365]



1it [00:27, 27.47s/it]

Best_fitness: [0.20387701]



2it [00:50, 26.22s/it]

...


18it [11:51, 39.40s/it]

Best_fitness: [0.05395857]



19it [12:32, 39.67s/it]

Best_fitness: [0.05814223]



20it [13:11, 39.57s/it]
4it [42:14, 633.52s/it]


In [8]:
       
for i, dataset in tqdm(enumerate(datasets)):
    X_train, y_train = dataset.iloc[:,:-1].to_numpy(), dataset.iloc[:,1].to_numpy()

    splitted_X_train, splitted_y_train = shuffle_and_split(X_train, y_train, batch_size=20)
    
    perfs[list(perfs.keys())[i]]["batch_size:20 swarmsize:200 itercount:200"] = []
    network = nets[i]
    for (x_batch,y_batch) in tqdm(zip(splitted_X_train,splitted_y_train)):
        perf = network.train_with_pso(x_batch, y_batch, swarm_size=200, iter_count=200)
        perfs[list(perfs.keys())[i]]["batch_size:20 swarmsize:200 itercount:200"].append(perf)
 

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Best_fitness: [0.2547329]



1it [08:37, 517.77s/it]

Best_fitness: [0.19849738]



2it [16:18, 500.54s/it]

...


18it [3:59:11, 791.75s/it]

Best_fitness: [0.03445244]



19it [4:12:28, 793.42s/it]

Best_fitness: [0.06544985]



20it [4:25:36, 796.83s/it]
4it [14:07:08, 12707.22s/it]


In [14]:
for i, dataset in tqdm(enumerate(datasets)):
    print(ONE_IN_FILES[i])
    network = nets[i]
    X_train, y_train = dataset.iloc[:,:-1].to_numpy(), dataset.iloc[:,1].to_numpy()
    print(network.test(X_train, y_train))
    print(network.network)

0it [00:00, ?it/s]

data/1in_tanh.txt
0.3105594808220541
{'weights': {'w1': array([[1.29959325]])}, 'biases': {'b1': array([[0.40102346]])}, 'activationFunctions': {'a1': 'arctan'}, 'fp_activation': {'w1': array([[1.03923075]])}, 'deltas': {}}
data/1in_linear.txt
0.010075521452264546
{'weights': {'w1': array([[0.97585357]])}, 'biases': {'b1': array([[0.01854913]])}, 'activationFunctions': {'a1': 'prelu'}, 'fp_activation': {'w1': array([[0.97488562]])}, 'deltas': {}}
data/1in_cubic.txt
0.08386435972999291
{'weights': {'w1': array([[0.61622556],
       [0.73985546],
       [0.85676743]]), 'w2': array([[0.62846182, 0.88063874, 0.95142929]])}, 'biases': {'b1': array([[0.72151288],
       [0.76771017],
       [0.7278703 ]]), 'b2': array([[0.68174396]])}, 'activationFunctions': {'a1': 'cosine', 'a2': 'gaussian'}, 'fp_activation': {'w1': array([[ 0.23095381],
       [ 0.06318857],
       [-0.01384096]]), 'w2': array([[0.68529925]])}, 'deltas': {}}
data/1in_sine.txt
0.13163535141178523
{'weights': {'w1': array([[

4it [00:00, 66.81it/s]


In [25]:
result_networks

{'1in_tan': {'shallow': {'model': <classes.network.Network at 0x1a007eac198>,
   'train_perfs': [[0.23799435735371297,
     0.23799435735371297,
     0.23799435735371297,
     0.23799435735371297,
     0.23799435735371297,
     0.23799435735371297,
     0.23799435735371297,
     0.2345112827552037,
     0.23420057472987676,
     0.23225096603977666],
    [0.3216436849409027,
     0.2883782805800687,
     0.2883782805800687,
     0.2828769479879259,
     0.2749441157728493,
     0.2726247518298349,
     0.2726247518298349,
     0.2726247518298349,
     0.2726247518298349,
     0.2719360983335553],
    [0.5460881126219124,
     0.46686170595130505,
     0.4622064633957046,
     0.4622064633957046,
     0.4622064633957046,
     0.4622064633957046,
     0.4622064633957046,
     0.4569856620050011,
     0.449812533399563,
     0.449812533399563],
    [0.18739028702453192,
     0.1563423213783648,
     0.12766255552967637,
     0.1144870905373161,
     0.1144870905373161,
     0.114487090537

In [21]:
import pickle

In [23]:
# Step 2
with open('results_experiments_one.dictionary', 'wb') as config_dictionary_file:
 
    # Step 3
    pickle.dump(result_networks, config_dictionary_file)